In [1]:
import os
import numpy as np
import pandas as pd
import scanpy as sc
from natsort import natsorted

import sys

In [2]:
dir_out = "/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome_ai/barcodes_l23"
# !mkdir -p $dir_out

In [3]:
%%time
f_rna = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/L23_allmultiome_proc_P6toP21_NRDR.h5ad'
adata = sc.read(f_rna, backed='r')
adata

CPU times: user 17.1 ms, sys: 2.41 s, total: 2.43 s
Wall time: 2.62 s


AnnData object with n_obs × n_vars = 38539 × 16509 backed at '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/L23_allmultiome_proc_P6toP21_NRDR.h5ad'
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'Age', 'Doublet', 'Doublet.Score', 'n_counts', 'n_genes', 'percent_mito', 'sample', 'Type', 'Subclass', 'Class', 'Sample', 'total_counts', 'pct_counts_mt', 'n_genes_by_counts', 'total_counts_mt', 'Doublet.', 'Study', 'Type_leiden', 'time', 'cond', 'easitype', 'cond_order'
    var: 'features'
    obsm: 'pca_p17on', 'pca_p8'
    layers: 'norm'

In [4]:
adata.obs.index

Index(['CTTAACAAGGAAGCTA-1-P6a-2023 Multiome-0-0',
       'GATCAAGAGTTACCGG-1-P6b-2023 Multiome-0-0',
       'ACCGAAGCATAATGAG-1-P6c-2023 Multiome-0-0',
       'AAGCGGGTCCACCTGT-1-P6c-2023 Multiome-0-0',
       'TGCTAGCCACATGCTA-1-P6b-2023 Multiome-0-0',
       'GGTCAGGAGCTTCCCG-1-P6c-2023 Multiome-0-0',
       'GTCCATTGTGAACAAA-1-P6c-2023 Multiome-0-0',
       'ATCGAGGCATATTGAC-1-P6a-2023 Multiome-0-0',
       'ATTTGTGAGTCATGCG-1-P6c-2023 Multiome-0-0',
       'CTGTAACAGGATCACT-1-P6c-2023 Multiome-0-0',
       ...
       'CTTACCGGTAATAGCT-1-P21DRb-2023 Multiome-10-0',
       'TTTCGTCCAAGCTTAT-1-P21DRa-2023 Multiome-10-0',
       'ATGAATGCAATTATGC-1-P21DRa-2023 Multiome-10-0',
       'CCCTGTTAGTTTCCGC-1-P21DRa-2023 Multiome-10-0',
       'GGTGAGTAGAAAGCAT-1-P21DRa-2023 Multiome-10-0',
       'CTCACACTCCCGTTTA-1-P21DRa-2023 Multiome-10-0',
       'TAATGGTGTGGAAGGC-1-P21DRb-2023 Multiome-10-0',
       'AGCATCCCATGCATAT-1-P21DRa-2023 Multiome-10-0',
       'GGTCTTTGTTTACTTG-1-P21DRb-2023 

In [5]:
cellname_breakdowns = np.array(list(adata.obs.index.str.split('-')))
cellname_breakdowns

array([['CTTAACAAGGAAGCTA', '1', 'P6a', '2023 Multiome', '0', '0'],
       ['GATCAAGAGTTACCGG', '1', 'P6b', '2023 Multiome', '0', '0'],
       ['ACCGAAGCATAATGAG', '1', 'P6c', '2023 Multiome', '0', '0'],
       ...,
       ['AGCATCCCATGCATAT', '1', 'P21DRa', '2023 Multiome', '10', '0'],
       ['GGTCTTTGTTTACTTG', '1', 'P21DRb', '2023 Multiome', '10', '0'],
       ['AAAGCTTGTTTGTCTA', '1', 'P21DRa', '2023 Multiome', '10', '0']],
      dtype='<U16')

### decode cellname 
- `longname = f"{barcode}-1-{sample}-2023 Multiome-{condition_code}-0`
- `shortname = f'{barcode}_{sample}_v1multiome'`

- barcode: ATGC....
- sample: P21a, P21b, P21DRa, P21DRb
- data: v1multiome

In [6]:
barcodes_vec = cellname_breakdowns[:,0]
samples_vec  = cellname_breakdowns[:,2]
shortnames = np.array([f"{i}_{j}_v1multiome" for i,j in zip(barcodes_vec, samples_vec)])
shortnames

array(['CTTAACAAGGAAGCTA_P6a_v1multiome',
       'GATCAAGAGTTACCGG_P6b_v1multiome',
       'ACCGAAGCATAATGAG_P6c_v1multiome', ...,
       'AGCATCCCATGCATAT_P21DRa_v1multiome',
       'GGTCTTTGTTTACTTG_P21DRb_v1multiome',
       'AAAGCTTGTTTGTCTA_P21DRa_v1multiome'], dtype='<U34')

In [7]:
uniq_samples = natsorted(np.unique(samples_vec))
uniq_samples

['P6a',
 'P6b',
 'P6c',
 'P8a',
 'P8b',
 'P8c',
 'P10a',
 'P10b',
 'P12DRa',
 'P12DRb',
 'P12a',
 'P12b',
 'P12c',
 'P14DRa',
 'P14DRb',
 'P14a',
 'P14b',
 'P17DRa',
 'P17DRb',
 'P17a',
 'P17b',
 'P21DRa',
 'P21DRb',
 'P21a',
 'P21b']

# per sample barcode whitelist

In [10]:
for sample in uniq_samples:
    sel = samples_vec == sample
    shortnames_subset = shortnames[sel]
    barcodes_vec_subset = barcodes_vec[sel]
    # print(shortnames_subset)
    # print(barcodes_vec_subset)
    print(sample, len(barcodes_vec_subset))
    
    fout = os.path.join(dir_out, f'barcodes_l23_{sample}.txt')
    print(fout)
    np.savetxt(fout, barcodes_vec_subset, '%s')
    # break

P6a 1521
/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome_ai/barcodes_l23/barcodes_l23_P6a.txt
P6b 1677
/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome_ai/barcodes_l23/barcodes_l23_P6b.txt
P6c 3414
/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome_ai/barcodes_l23/barcodes_l23_P6c.txt
P8a 1762
/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome_ai/barcodes_l23/barcodes_l23_P8a.txt
P8b 1787
/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome_ai/barcodes_l23/barcodes_l23_P8b.txt
P8c 1588
/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome_ai/barcodes_l23/barcodes_l23_P8c.txt
P10a 1723
/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome_ai/barcodes_l23/barcodes_l23_P10a.txt
P10b 1412
/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome_ai/barcodes_l23/barcodes_l23_P10b.txt
P12DRa 1322
/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome_ai/barcodes_l23/barcodes_l23_P12DRa.txt
P12DRb 1460
/u/hom

In [11]:
!wc -l $fout 
!head $fout

1020 /u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome_ai/barcodes_l23/barcodes_l23_P21b.txt
AAACCGAAGTTGCGTG
AAAGCACCAGTAAAGC
AAAGGTTAGAGGGACT
AAATGCCTCGGTCAGC
AACAGATAGGCATTGT
AACAGATAGTCACCAG
AACCCGCAGAGGAAGG
AACCCGCAGTAACCAC
AACGACAAGGCCAATT
AACGCCCAGTTTGAGC
